# Cleanup Missing Data

Dieses Notebook liest alle Bildernamen im 256x256 Ordner und löscht alle Daten, die nicht in diesem Ordner vorhanden sind:
- Bilder in 512x512 und 1024x1024 Ordnern
- Entsprechende Zeilen in den CSV-Dateien

In [1]:
import os
import pandas as pd
import glob
from pathlib import Path

# Pfade definieren
base_path = Path('../output')
images_path = base_path / 'images'
data_path = base_path / 'data'

print(f"Base path: {base_path}")
print(f"Images path: {images_path}")
print(f"Data path: {data_path}")

Base path: ..\output
Images path: ..\output\images
Data path: ..\output\data


In [2]:
# Alle Bilder im 256x256 Ordner lesen
reference_folder = images_path / '256x256'
reference_images = list(reference_folder.glob('*.jpg'))
reference_ids = [img.stem for img in reference_images]  # Dateiname ohne Extension

print(f"Anzahl Bilder in 256x256: {len(reference_ids)}")
print(f"Erste 10 IDs: {reference_ids[:10]}")

Anzahl Bilder in 256x256: 746
Erste 10 IDs: ['logowiz000002', 'logowiz000006', 'logowiz000007', 'logowiz000008', 'logowiz000009', 'logowiz000012', 'logowiz000013', 'logowiz000014', 'logowiz000015', 'logowiz000018']


In [3]:
# Funktion zum Löschen von Bildern in anderen Ordnern
def cleanup_image_folders(reference_ids, folders_to_clean):
    deleted_count = 0
    
    for folder in folders_to_clean:
        folder_path = images_path / folder
        if not folder_path.exists():
            print(f"Ordner {folder_path} existiert nicht")
            continue
            
        # Alle Bilder in diesem Ordner finden
        all_images = list(folder_path.glob('*.jpg'))
        
        for img_path in all_images:
            img_id = img_path.stem
            
            # Löschen wenn ID nicht in Referenz-Liste
            if img_id not in reference_ids:
                print(f"Lösche {img_path}")
                img_path.unlink()
                deleted_count += 1
        
        print(f"Ordner {folder}: {len(all_images)} Bilder überprüft")
    
    print(f"\nInsgesamt {deleted_count} Bilder gelöscht")
    return deleted_count

# Bilder in 512x512 und 1024x1024 bereinigen
folders_to_clean = ['512x512', '1024x1024']
deleted_images = cleanup_image_folders(reference_ids, folders_to_clean)

Lösche ..\output\images\512x512\logowiz000000.jpg
Lösche ..\output\images\512x512\logowiz000001.jpg
Lösche ..\output\images\512x512\logowiz000003.jpg
Lösche ..\output\images\512x512\logowiz000004.jpg
Lösche ..\output\images\512x512\logowiz000005.jpg
Lösche ..\output\images\512x512\logowiz000010.jpg
Lösche ..\output\images\512x512\logowiz000011.jpg
Lösche ..\output\images\512x512\logowiz000016.jpg
Lösche ..\output\images\512x512\logowiz000017.jpg
Lösche ..\output\images\512x512\logowiz000020.jpg
Lösche ..\output\images\512x512\logowiz000022.jpg
Lösche ..\output\images\512x512\logowiz000023.jpg
Lösche ..\output\images\512x512\logowiz000024.jpg
Lösche ..\output\images\512x512\logowiz000026.jpg
Lösche ..\output\images\512x512\logowiz000028.jpg
Lösche ..\output\images\512x512\logowiz000029.jpg
Lösche ..\output\images\512x512\logowiz000031.jpg
Lösche ..\output\images\512x512\logowiz000035.jpg
Lösche ..\output\images\512x512\logowiz000039.jpg
Lösche ..\output\images\512x512\logowiz000041.jpg


In [4]:
# CSV-Dateien finden und bereinigen
def cleanup_csv_files(reference_ids, data_path):
    csv_files = list(data_path.glob('*_logo_*.csv'))
    
    for csv_file in csv_files:
        print(f"\nBearbeite {csv_file.name}")
        
        # CSV laden
        df = pd.read_csv(csv_file)
        original_count = len(df)
        print(f"Ursprüngliche Anzahl Zeilen: {original_count}")
        
        # ID-Spalte finden (könnte 'id', 'image_id', etc. sein)
        id_column = None
        for col in df.columns:
            if 'id' in col.lower() or col in ['filename', 'name']:
                id_column = col
                break
        
        if id_column is None:
            print(f"Keine ID-Spalte in {csv_file.name} gefunden. Spalten: {list(df.columns)}")
            continue
            
        print(f"Verwende Spalte '{id_column}' als ID")
        
        # IDs extrahieren (falls Dateiendungen enthalten sind, entfernen)
        df['clean_id'] = df[id_column].astype(str).str.replace('.jpg', '').str.replace('.jpeg', '')
        
        # Nur Zeilen behalten, deren IDs in der Referenz-Liste sind
        df_filtered = df[df['clean_id'].isin(reference_ids)]
        df_filtered = df_filtered.drop('clean_id', axis=1)  # Hilfsspalte entfernen
        
        filtered_count = len(df_filtered)
        deleted_rows = original_count - filtered_count
        
        print(f"Gefilterte Anzahl Zeilen: {filtered_count}")
        print(f"Gelöschte Zeilen: {deleted_rows}")
        
        # Gefilterte CSV speichern
        df_filtered.to_csv(csv_file, index=False)
        print(f"Datei {csv_file.name} aktualisiert")

# CSV-Dateien bereinigen
cleanup_csv_files(reference_ids, data_path)


Bearbeite logowiz_logo_bytes.csv
Ursprüngliche Anzahl Zeilen: 803
Verwende Spalte 'unique_id' als ID
Gefilterte Anzahl Zeilen: 746
Gelöschte Zeilen: 57
Datei logowiz_logo_bytes.csv aktualisiert

Bearbeite logowiz_logo_metadata.csv
Ursprüngliche Anzahl Zeilen: 803
Verwende Spalte 'unique_id' als ID
Gefilterte Anzahl Zeilen: 746
Gelöschte Zeilen: 57
Datei logowiz_logo_metadata.csv aktualisiert


In [5]:
# Zusammenfassung der Bereinigung
print("\n=== BEREINIGUNG ABGESCHLOSSEN ===")
print(f"Referenz: {len(reference_ids)} Bilder in 256x256")

# Aktuelle Anzahl in anderen Ordnern prüfen
for folder in ['512x512', '1024x1024']:
    folder_path = images_path / folder
    if folder_path.exists():
        current_count = len(list(folder_path.glob('*.jpg')))
        print(f"{folder}: {current_count} Bilder verbleibend")

# CSV-Dateien Anzahl prüfen
csv_files = list(data_path.glob('*_logo_*.csv'))
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    print(f"{csv_file.name}: {len(df)} Zeilen verbleibend")


=== BEREINIGUNG ABGESCHLOSSEN ===
Referenz: 746 Bilder in 256x256
512x512: 746 Bilder verbleibend
1024x1024: 746 Bilder verbleibend
logowiz_logo_bytes.csv: 746 Zeilen verbleibend
logowiz_logo_metadata.csv: 746 Zeilen verbleibend
